# Import Data and Packages

In [55]:
import pandas as pd
import numpy as np

In [56]:
trn = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Combine Dataframes to Clean Together

In [3]:
# We can use this information to split the two after we've treated them together
print(max(trn.Id))
print(min(test.Id))

1460
1461


In [57]:
test['SalePrice']=0

In [58]:
df = pd.concat([trn,test])

# View Missingness

In [77]:
total = df.isnull().sum().sort_values(ascending=False)
percent = 100*(df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data = missing_data.round(2)
missing_data[missing_data.Total>0]

,Total,Percent
PoolQC,2909,99.66
MiscFeature,2814,96.40
Alley,2721,93.22
Fence,2348,80.44
FireplaceQu,1420,48.65
LotFrontage,486,16.65
GarageCond,159,5.45
GarageYrBlt,159,5.45
GarageFinish,159,5.45
GarageQual,159,5.45


In [7]:
# remove these variables
df = df.drop(['Utilities','PoolQC','MiscFeature','Fence'],axis=1)

In [8]:
# convert alley NA to seperate category
df.loc[df.Alley.isnull(),'Alley']='None'

In [9]:
# functional
df.loc[df.Functional.isnull(),'Functional']='Typ'
df = df.replace({"Functional" : {"Typ" : 7,
    "Min1" : 6,
    "Min2" : 5,
    "Mod" : 4,
    "Maj1" : 3,
    "Maj2" : 2,
    "Sev" : 1}})

In [10]:
# heatingQC
df = df.replace({"HeatingQC" : {"Ex" : 5, "Gd" : 4, "TA" : 3, "Fa" : 2, "Po" : 1}})

In [11]:
# exterqual and extercond
df = df.replace({"ExterQual" : {"Ex" : 5, "Gd" : 4, "TA" : 3, "Fa" : 2, "Po" : 1}})

df = df.replace({"ExterCond" : {"Ex" : 5, "Gd" : 4, "TA" : 3, "Fa" : 2, "Po" : 1}})

In [12]:
# fireplace quality - ordinal categorical with NAs = 0
df = df.replace({"FireplaceQu" : {"Ex" : 5, "Gd" : 4, "TA" : 3, "Fa" : 2, "Po" : 1}})
df['FireplaceQu'] = df['FireplaceQu'].fillna(0)

In [13]:
# kitchen qual
df = df.replace({"KitchenQual" : {"Ex" : 5, "Gd" : 4, "TA" : 3, "Fa" : 2, "Po" : 1}})
df['KitchenQual'] = df['KitchenQual'].fillna(0)

In [14]:
# lot frontage impute 0s
df['LotFrontage'] = df['LotFrontage'].fillna(0)

In [59]:
# convert the two integer-type categories to categories
df = df.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45",
                                       50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75",
                                       80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120",
                                       150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},
                       "MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
                                   7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
                      })

In [16]:
# Garage variables

df.loc[df.GarageType.isnull(),'GarageType']='None'
#change GarageFinish to be ordinal with NAs = 0
df = df.replace({"GarageFinish" : {"Fin" : 3, "RFn" : 2, "Unf" : 1}})
df['GarageFinish'] = df['GarageFinish'].fillna(0)
# change GarageCond to be ordinal with NAs = 0
df = df.replace({"GarageCond" : {"Ex" : 5, "Gd" : 4, "TA" : 3, "Fa" : 2, "Po" : 1}})
df['GarageCond'] = df['GarageCond'].fillna(0)
# change GarageQual to be ordinal with NAs = 0
df = df.replace({"GarageQual" : {"Ex" : 5, "Gd" : 4, "TA" : 3, "Fa" : 2, "Po" : 1}})
df['GarageQual'] = df['GarageQual'].fillna(0)
#drop Garage yr blt
df = df.drop('GarageYrBlt', axis=1)

In [17]:
# Basement variables:

# DROP BsmtFinType(s) because I can't figure out how to deal with them
df = df.drop(['BsmtFinType1'],axis=1)
df = df.drop(['BsmtFinType2'],axis=1)
# BsmtQual ord
df = df.replace({"BsmtQual" : {"Ex" : 5, "Gd" : 4, "TA" : 3, "Fa" : 2, "Po" : 1}})
df['BsmtQual'] = df['BsmtQual'].fillna(0)
# BsmtCond 
df = df.replace({"BsmtCond" : {"Ex" : 5, "Gd" : 4, "TA" : 3, "Fa" : 2, "Po" : 1}})
df['BsmtCond'] = df['BsmtCond'].fillna(0)
# BsmtExposure
df = df.replace({"BsmtExposure" : {"Gd" : 4, "Av" : 3, "Mn" : 2, "No" : 1}})
df['BsmtExposure'] = df['BsmtExposure'].fillna(0)

In [18]:
# Mason variables:

df.loc[df.MasVnrType.isnull(),'MasVnrType']='None'
df.loc[df.MasVnrArea.isnull(),'MasVnrArea']=0

In [19]:
# Electrical - impute 1 NA to be most commonly represented category (by comfortable margin)
df.loc[df.Electrical.isnull(),'Electrical']='SBrkr'

In [20]:
# ASSUME MSZoning is equal to RL
df.loc[df.MSZoning.isnull(),'MSZoning']='RL'

In [21]:
# ASSUME Functional is Typ
df.loc[df.Functional.isnull(), 'Functional']='Typ'

In [22]:
# ASSUME null means 0 bsmt baths
df.loc[df.BsmtFullBath.isnull(), 'BsmtFullBath']=0.0
df.loc[df.BsmtHalfBath.isnull(), 'BsmtHalfBath']=0.0

In [23]:
# find that bsmt sf should be 0
df.loc[df.TotalBsmtSF.isnull(),'BsmtQual']
df.loc[df.TotalBsmtSF.isnull(),'TotalBsmtSF']=0

df.loc[df.BsmtFinSF2.isnull(),'BsmtQual']
df.loc[df.BsmtFinSF2.isnull(),'BsmtFinSF2']=0

df.loc[df.BsmtFinSF1.isnull(),'BsmtQual']
df.loc[df.BsmtFinSF1.isnull(),'BsmtFinSF1']=0

df.loc[df.BsmtUnfSF.isnull(),'BsmtQual']
df.loc[df.BsmtUnfSF.isnull(),'BsmtUnfSF']=0

In [24]:
# ASSUME SaleType is most common
df.loc[df.SaleType.isnull(),'SaleType']='WD'

In [25]:
# find that garage cars/area should be 0
df.loc[df.GarageCars.isnull(),'GarageCars']=0
df.loc[df.GarageArea.isnull(),'GarageArea']=0

In [26]:
# ASSUME exteriors are most common
df.Exterior2nd.value_counts()
df.loc[df.Exterior1st.isnull(),'Exterior1st']='VinylSd'
df.loc[df.Exterior2nd.isnull(),'Exterior2nd']='VinylSd'

# Feature Engineering

In [27]:
# adding Total Square Foot variable
df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']

In [28]:
# drop the above
df = df.drop(['TotalBsmtSF','1stFlrSF','2ndFlrSF'],axis=1)

In [29]:
# combining outdoor space
df['OutdoorSF']=df.WoodDeckSF+df.OpenPorchSF+df.EnclosedPorch+df['3SsnPorch']+df.ScreenPorch

In [30]:
# drop the above
df = df.drop(['WoodDeckSF','OpenPorchSF','3SsnPorch','EnclosedPorch','ScreenPorch'],axis=1)

In [31]:
# create dummies
df = pd.get_dummies(df)

# Split Combined Dataframe

In [32]:
trn = df[:1460]
test = df[1460:]

In [33]:
test = test.drop('SalePrice',axis=1)

In [75]:
len(df.columns)

81

# Save to New Files

In [36]:
trn.to_csv('cleantrain.csv')
test.to_csv('cleantest.csv')